# Libraries

In [1]:
# import pandas as pd
# import numpy as np
# import time
# import twython
# import requests
# import nltk
# import warnings
# warnings.filterwarnings('ignore')

# from htmldate import find_date
# from tqdm import tqdm
# from bs4 import BeautifulSoup
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.downloader.download('vader_lexicon')
# from newspaper import Article

In [8]:
import pandas as pd
import re
import time
from htmldate import find_date
from newspaper import Article
from bs4 import BeautifulSoup
import requests
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\truon\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [56]:
def download_news(urls, sentiment_analysis=True):
    if sentiment_analysis:
        articles_df = pd.DataFrame({'publish_date':[], 'title': [],'body_text': [], 'url':[],
                                    'neg':[], 'neu':[], 'pos':[], 'compound':[]})
    else:
        articles_df = pd.DataFrame({'publish_date':[], 'title': [],'body_text': [], 'url':[]})  
                                
    for link in tqdm(urls):
        article = Article(link)
        article.download()
        
        try:
            article.parse()
            text = article.text

        except: 
            print("I didn't get this")
            continue

        try:
            date = find_date(link)
        except:
            print("Failed to get date.")
            continue

        tmpdic = {'publish_date': date, 'title': article.title, 'body_text': text, 'url': link}

        if sentiment_analysis:
            # Initialise sentiment analyser  
            sid = SentimentIntensityAnalyzer()
            # Get positive, negative, neutral and compound scores
            polarity = sid.polarity_scores(text)
            # Update ticker with the new entry polarity
            tmpdic.update(polarity)
        
        # tmpdic now has all keys and values needed to populate the DataFrame
        articles_df.loc[articles_df.shape[0]] = tmpdic

    return articles_df

In [29]:
def search_for_raw_urls(ticker, page):
    raw_urls = []
    for i in page:
        search_url = f"https://www.google.com/search?q=yahoo+finance+{ticker}&tbm=nws&start={i}"
        r = requests.get(search_url)
        soup = BeautifulSoup(r.text, 'html.parser')
        atags = soup.find_all('a')
        hrefs = [link['href'] for link in atags]
        raw_urls.extend(hrefs)
        time.sleep(5)
    return raw_urls

def clean_urls(urls, exclude_list=['maps', 'policies', 'preferences', 'accounts', 'support'],
                        include_list=["https://finance.yahoo.com/news/"]):
    val = []
    for url in urls:
        if any(exc in url for exc in include_list) and not any(exc in url for exc in exclude_list):
            res = re.findall(r'(https?://\S+html)', url)[0]
            val.append(res)
    return list(set(val))

def get_news_urls(ticker, page, exclude_list=['maps', 'policies', 'preferences', 'accounts', 'support'],
                  include_list=["https://finance.yahoo.com/news/"]):
    raw_urls = search_for_raw_urls(ticker, page)
    cleaned_urls = clean_urls(raw_urls, exclude_list, include_list)
    return cleaned_urls

In [48]:
raw_urls = search_for_raw_urls("aapl", page=range(0, 105, 5))
raw_urls

['/?sa=X&ved=0ahUKEwih4cXth-2FAxVUklYBHeniCmEQOwgC',
 '/search?q=yahoo+finance+aapl&sca_esv=b511c58466c4623d&ie=UTF-8&tbm=nws&gbv=1&sei=FYcyZuGCBdSk2roP6cWriAY',
 '/search?q=yahoo+finance+aapl&sca_esv=b511c58466c4623d&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwih4cXth-2FAxVUklYBHeniCmEQ_AUIBSgA',
 '/search?q=yahoo+finance+aapl&sca_esv=b511c58466c4623d&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwih4cXth-2FAxVUklYBHeniCmEQ_AUIBygC',
 '/search?q=yahoo+finance+aapl&sca_esv=b511c58466c4623d&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwih4cXth-2FAxVUklYBHeniCmEQ_AUICCgD',
 '/search?q=yahoo+finance+aapl&sca_esv=b511c58466c4623d&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwih4cXth-2FAxVUklYBHeniCmEQ_AUICSgE',
 '/url?q=/search%3Fq%3Dyahoo%2Bfinance%2Baapl%26sca_esv%3Db511c58466c4623d%26ie%3DUTF-8%26tbm%3Dshop%26source%3Dlnms%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwih4cXth-2FAxVUklYBHeniCmEQiaAMCAooBQ&usg=AOvVaw3VcGOTDt1FO_Eu4v2km1kB',
 '/advanced_search',
 '/search?q=yahoo+finan

In [51]:
raw_urls = ['https://finance.yahoo.com/news/hold-apple-aapl-long-term-123749460.html',
 'https://finance.yahoo.com/news/apple-best-magnificent-seven-stock-113000731.html',
 'https://finance.yahoo.com/news/dare-invest-apple-aapl-064949251.html',
 'https://finance.yahoo.com/news/apple-stock-down-10-5-224048496.html',
 'https://finance.yahoo.com/news/apples-ipados-designated-gatekeeper-under-095015330.html',
 'https://finance.yahoo.com/news/5-apple-products-avoid-buying-140009943.html',
 'https://finance.yahoo.com/news/apple-signal-ai-intent-unleashes-114945606.html',
 'https://finance.yahoo.com/news/trending-stock-apple-inc-aapl-130018800.html',
 'https://finance.yahoo.com/news/apple-stock-could-notch-record-194747011.html',
 'https://finance.yahoo.com/news/2-2-billion-reasons-buy-144500461.html',
 'https://finance.yahoo.com/news/apples-glass-supplier-corning-outperforms-194542581.html',
 'https://finance.yahoo.com/news/too-buy-apple-stock-110900320.html',
 'https://finance.yahoo.com/news/where-apple-stock-2025-090000324.html',
 'https://finance.yahoo.com/news/forget-apple-buy-unstoppable-growth-150000634.html',
 'https://finance.yahoo.com/news/unfortunate-news-meta-stock-apple-093000537.html',
 'https://finance.yahoo.com/news/analyst-upgrades-apple-stock-earnings-185619605.html',
 'https://finance.yahoo.com/news/apple-stock-top-holding-tech-120000467.html',
 'https://finance.yahoo.com/news/apples-first-quarter-has-felt-more-like-an-entire-bad-year-201715880.html',
 'https://finance.yahoo.com/news/apple-trillion-dollar-stock-2035-100000749.html',
 'https://finance.yahoo.com/news/apples-vision-pro-lands-as-the-company-faces-threats-from-all-sides-225650277.html',
 'https://finance.yahoo.com/news/apple-amazon-earnings-tech-stocks-100000361.html',
 'https://finance.yahoo.com/news/apple-stock-soar-36-gears-232719784.html',
 'https://finance.yahoo.com/news/apple-stock-slips-as-company-prepares-to-halt-apple-watch-sales-224142282.html',
 'https://finance.yahoo.com/news/happen-warren-buffett-bailed-apple-172823491.html',
 'https://finance.yahoo.com/news/buy-apple-aapl-stock-ahead-164700880.html',
 'https://finance.yahoo.com/news/stock-market-today-nasdaq-surges-as-chipmakers-shine-apple-pops-210312922.html',
 'https://finance.yahoo.com/news/too-buy-apple-stock-101500323.html',
 'https://finance.yahoo.com/news/apple-inc-aapl-ceo-timothy-055354884.html',
 'https://finance.yahoo.com/news/apple-named-top-pick-2024-150637860.html',
 'https://finance.yahoo.com/news/american-politicians-buying-10-ai-215904213.html',
 'https://finance.yahoo.com/news/apple-aapl-stock-sinks-market-214520350.html',
 'https://finance.yahoo.com/news/apple-stock-sinks-despite-earnings-beat-as-china-sales-slow-144509608.html',
 'https://finance.yahoo.com/news/trending-stock-apple-inc-aapl-140010235.html',
 'https://finance.yahoo.com/news/sun-set-apples-innovation-driven-142100071.html',
 'https://finance.yahoo.com/news/its-not-all-doom-and-gloom-for-apple-in-2024-202332782.html',
 'https://finance.yahoo.com/news/buy-apple-inc-nasdaq-aapl-110103890.html',
 'https://finance.yahoo.com/news/apple-strengthens-supply-chain-china-152635915.html',
 'https://finance.yahoo.com/news/apple-stock-faces-three-downgrades-but-not-everyone-is-down-on-the-company-192846428.html',
 'https://finance.yahoo.com/news/apple-stock-nasdaq-aapl-2-055048576.html',
 'https://finance.yahoo.com/news/apple-stock-27-upside-according-171259933.html',
 'https://finance.yahoo.com/news/analyst-revamps-apple-stock-rating-132103626.html',
 'https://finance.yahoo.com/news/apple-aapl-beat-estimates-again-161014112.html',
 'https://finance.yahoo.com/news/apple-aapl-registers-bigger-fall-214522452.html',
 'https://finance.yahoo.com/news/apple-stock-31-upside-according-133313699.html',
 'https://finance.yahoo.com/news/apple-aapl-falls-more-steeply-214518929.html',
 'https://finance.yahoo.com/news/apple-stock-27-upside-according-201112216.html',
 'https://finance.yahoo.com/news/apple-aapl-advances-underperforms-market-224518956.html',
 'https://finance.yahoo.com/news/apple-looks-to-southeast-asia-india-as-hedge-against-china-difficulties-185416388.html',
 'https://finance.yahoo.com/news/2-reasons-buy-apple-stock-073000374.html',
 'https://finance.yahoo.com/news/prediction-warren-buffetts-second-biggest-095000279.html',
 'https://finance.yahoo.com/news/apple-aapl-rises-higher-market-224513451.html',
 'https://finance.yahoo.com/news/apple-ceo-tim-cook-much-170220216.html',
 'https://finance.yahoo.com/news/apple-earnings-time-bite-192900665.html',
 'https://finance.yahoo.com/news/apple-stock-buy-221100899.html',
 'https://finance.yahoo.com/news/paypal-raises-annual-outlook-expands-153301289.html',
 'https://finance.yahoo.com/news/google-apple-now-threatened-us-194925838.html',
 'https://finance.yahoo.com/news/apple-stock-nasdaq-aapl-could-055803122.html',
 'https://finance.yahoo.com/news/apple-aapl-ascends-while-market-224522610.html',
 'https://finance.yahoo.com/news/earn-500-month-apple-stock-150245785.html',
 'https://finance.yahoo.com/news/zacks-market-edge-highlights-aapl-083500150.html',
 'https://finance.yahoo.com/news/earn-500-month-apple-stock-170539964.html',
 'https://finance.yahoo.com/news/where-apple-stock-3-years-075900640.html',
 'https://finance.yahoo.com/news/meet-unstoppable-stock-just-joined-151500438.html',
 'https://finance.yahoo.com/news/buy-apple-stock-earnings-204908376.html',
 'https://finance.yahoo.com/news/meta-platforms-bought-time-long-154934660.html',
 'https://finance.yahoo.com/news/where-apple-stock-3-years-075000500.html',
 'https://finance.yahoo.com/news/wall-street-analyst-warns-apple-163014714.html',
 'https://finance.yahoo.com/news/apple-renews-talks-openai-iphone-224431419.html',
 'https://finance.yahoo.com/news/apples-earnings-power-once-again-210654562.html',
 'https://finance.yahoo.com/news/down-over-10-time-buy-111500040.html',
 'https://finance.yahoo.com/news/where-apple-stock-5-years-145441853.html',
 'https://finance.yahoo.com/news/forget-apple-think-stock-replace-110900654.html',
 'https://finance.yahoo.com/news/apple-shares-rise-3-analyst-191312617.html',
 'https://finance.yahoo.com/news/apple-aapl-expected-beat-earnings-140211938.html',
 'https://finance.yahoo.com/news/apple-aapl-advances-while-market-214520651.html',
 'https://finance.yahoo.com/news/apple-3-trillion-stock-2025-151500285.html',
 'https://finance.yahoo.com/news/why-wall-street-is-cooling-on-apple-stock-194143216.html',
 'https://finance.yahoo.com/news/not-holding-apple-aapl-affected-130012483.html',
 'https://finance.yahoo.com/news/3-apple-products-earned-loud-194822048.html',
 'https://finance.yahoo.com/news/apple-is-aiming-the-vision-pro-at-enterprises-140056694.html',
 'https://finance.yahoo.com/news/reddit-apple-wetherspoons-vodafone-trending-tickers-094209950.html',
 'https://finance.yahoo.com/news/analysis-google-apple-breakups-agenda-010424313.html',
 'https://finance.yahoo.com/news/apple-best-magnificent-seven-stock-113000731.html',
 'https://finance.yahoo.com/news/apple-stock-nasdaq-aapl-vision-000011752.html',
 'https://finance.yahoo.com/news/meta-platforms-worth-more-apple-163000701.html',
 'https://finance.yahoo.com/news/dare-invest-apple-aapl-064949251.html',
 'https://finance.yahoo.com/news/apple-nasdaq-aapl-seems-debt-140013626.html',
 'https://finance.yahoo.com/news/apple-stock-down-36-as-barclays-says-iphone-weakness-will-persist-220019303.html',
 'https://finance.yahoo.com/news/microsoft-briefly-overtakes-apple-as-worlds-most-valuable-company-before-apple-takes-back-crown-165431534.html',
 'https://finance.yahoo.com/news/fed-meeting-apple-earnings-what-to-know-this-week-140035960.html',
 'https://finance.yahoo.com/news/heres-why-apple-stock-climbed-211749314.html',
 'https://finance.yahoo.com/news/microsoft-edges-apple-worlds-most-232740546.html',
 'https://finance.yahoo.com/news/apple-inc-aapl-attracting-investor-130015077.html',
 'https://finance.yahoo.com/news/2-2-billion-reasons-buy-144500461.html',
 'https://finance.yahoo.com/news/2-stocks-belong-magnificent-7-202800368.html',
 'https://finance.yahoo.com/news/warren-buffett-soon-begin-selling-105000506.html',
 'https://finance.yahoo.com/news/best-stock-buy-now-apple-124000378.html',
 'https://finance.yahoo.com/news/warren-buffett-bought-more-stock-122500638.html',
 'https://finance.yahoo.com/news/why-microsoft-stock-better-investment-120100900.html',
 'https://finance.yahoo.com/news/warren-buffett-sold-nearly-2-131026236.html',
 'https://finance.yahoo.com/news/wall-street-downgrading-apple-stock-105700079.html',
 'https://finance.yahoo.com/news/zacks-value-investor-highlights-apple-090400019.html',
 'https://finance.yahoo.com/news/apple-gets-pipped-uniqlo-lifts-054140662.html',
 'https://finance.yahoo.com/news/apple-could-have-a-new-problem-uncle-sam-115650004.html',
 'https://finance.yahoo.com/news/dr-congo-presses-apple-over-113858322.html',
 'https://finance.yahoo.com/news/top-stock-reports-apple-unitedhealth-190400751.html',
 'https://finance.yahoo.com/news/apple-ends-yearlong-sales-slump-215349301.html',
 'https://finance.yahoo.com/news/bofa-upgrades-apple-stock-to-buy-citing-vision-pro-and-generative-ai-as-growth-catalysts-182414468.html',
 'https://finance.yahoo.com/news/zacks-analyst-blog-highlights-apple-102000663.html',
 'https://finance.yahoo.com/news/golden-investment-opportunity-1-top-100000053.html',
 'https://finance.yahoo.com/news/microsoft-still-worth-more-apple-100500615.html',
 'https://finance.yahoo.com/news/apple-stock-nasdaq-aapl-siri-004643404.html',
 'https://finance.yahoo.com/news/apple-stock-fell-last-week-170159878.html',
 'https://finance.yahoo.com/news/move-over-apple-microsoft-now-224941997.html',
 'https://finance.yahoo.com/news/worth-investing-apple-aapl-based-143006530.html',
 'https://finance.yahoo.com/news/earn-500-month-apple-stock-161545270.html',
 'https://finance.yahoo.com/news/calculating-intrinsic-value-apple-inc-130019817.html',
 'https://finance.yahoo.com/news/microsoft-surpasses-apple-exceeds-3-104500030.html',
 'https://finance.yahoo.com/news/huaweis-gain-apples-pain-130158485.html',
 'https://finance.yahoo.com/news/earn-500-per-month-apple-172301452.html',
 'https://finance.yahoo.com/news/bull-day-apple-aapl-090000602.html',
 'https://finance.yahoo.com/news/once-generation-investment-opportunity-1-113000452.html',
 'https://finance.yahoo.com/news/dump-apple-following-buffett-etfs-180000934.html',
 'https://finance.yahoo.com/news/too-buy-apple-stock-101000706.html',
 'https://finance.yahoo.com/news/stocks-inch-up-as-apple-halts-slide-fed-officials-hint-at-rate-respite-stock-market-news-today-200213632.html',
 'https://finance.yahoo.com/news/asml-worth-more-apple-2030-081500048.html',
 'https://finance.yahoo.com/news/top-apple-analyst-warns-vision-132448619.html',
 'https://finance.yahoo.com/news/too-buy-apple-stock-173000102.html',
 'https://finance.yahoo.com/news/forget-apple-magnificent-seven-stock-103500633.html',
 'https://finance.yahoo.com/news/apple-stock-time-buy-dip-141300881.html',
 'https://finance.yahoo.com/news/4-measures-indicate-apple-nasdaq-120016269.html',
 'https://finance.yahoo.com/news/alphabet-stock-soaring-today-buy-172812858.html',
 'https://finance.yahoo.com/news/nvidia-cusp-overtaking-apple-second-160413392.html',
 'https://finance.yahoo.com/news/microsoft-topples-apple-become-global-083017494.html',
 'https://finance.yahoo.com/news/apple-stock-nasdaq-aapl-downgrade-184847890.html',
 'https://finance.yahoo.com/news/time-consider-buying-apple-inc-110027871.html',
 'https://finance.yahoo.com/news/mesa-community-college-adds-jamf-131500845.html',
 'https://finance.yahoo.com/news/buy-apple-inc-nasdaq-aapl-110103890.html',
 'https://finance.yahoo.com/news/apple-launches-vision-pro-headset-feb-2-preorders-begin-jan-19-155218836.html',
 'https://finance.yahoo.com/news/investors-buy-dip-apple-stock-114500340.html',
 'https://finance.yahoo.com/news/apple-inc-aapl-deep-dive-170333276.html',
 'https://finance.yahoo.com/news/could-warren-buffetts-shrewdest-big-105000230.html',
 'https://finance.yahoo.com/news/wall-street-analysts-look-bullish-133009293.html',
 'https://finance.yahoo.com/news/amid-rumors-deal-rivian-apple-113000766.html',
 'https://finance.yahoo.com/news/missed-apple-buy-essential-supplier-105500271.html',
 'https://finance.yahoo.com/news/justice-department-files-antitrust-suit-against-apple-145514025.html',
 'https://finance.yahoo.com/news/beyond-market-price-uncovering-apple-110011531.html',
 'https://finance.yahoo.com/news/earn-500-month-apple-stock-145705184.html',
 'https://finance.yahoo.com/news/apple-aapl-advances-underperforms-market-224518956.html',
 'https://finance.yahoo.com/news/invested-10-000-apple-tim-153000242.html',
 'https://finance.yahoo.com/news/jim-cramer-harsh-words-tim-193840560.html',
 'https://finance.yahoo.com/news/warren-buffetts-apple-inc-aapl-215409738.html',
 'https://finance.yahoo.com/news/horizon-kinetics-chinese-tech-boosted-112405942.html',
 'https://finance.yahoo.com/news/apple-stock-nasdaq-aapl-reality-202007973.html',
 'https://finance.yahoo.com/news/microsoft-challenges-apple-worlds-most-182711009.html',
 'https://finance.yahoo.com/news/marketmind-apple-angst-tesla-tanks-111114122.html',
 'https://finance.yahoo.com/news/apple-stock-sell-according-wall-113800855.html',
 'https://finance.yahoo.com/news/apple-stock-headed-250-143500711.html',
 'https://finance.yahoo.com/news/apples-app-store-rule-changes-draw-sharp-rebuke-from-critics-150047160.html',
 'https://finance.yahoo.com/news/apple-ceo-meet-singapore-leader-024614369.html',
 'https://finance.yahoo.com/news/apple-aapl-facing-increasing-risk-141649945.html',
 'https://finance.yahoo.com/news/could-apple-help-become-millionaire-101400281.html',
 'https://finance.yahoo.com/news/better-investment-10-years-ago-202750699.html',
 'https://finance.yahoo.com/news/apples-antitrust-fight-could-threaten-its-search-for-the-next-big-thing-143045864.html',
 'https://finance.yahoo.com/news/apples-nasdaq-aapl-five-earnings-120021930.html',
 'https://finance.yahoo.com/news/zacks-earnings-trends-highlights-apple-083700928.html',
 'https://finance.yahoo.com/news/apple-shares-wobble-antitrust-suit-133513878.html',
 'https://finance.yahoo.com/news/buy-apple-aapl-ahead-earnings-135000352.html',
 'https://finance.yahoo.com/news/top-stock-reports-apple-nvidia-183800969.html',
 'https://finance.yahoo.com/news/apple-slides-goldman-sachs-analysts-125504819.html',
 'https://finance.yahoo.com/news/apple-stock-buy-now-130000629.html',
 'https://finance.yahoo.com/news/apple-stock-buy-221100899.html',
 'https://finance.yahoo.com/news/apple-aapl-surpasses-market-returns-214518604.html',
 'https://finance.yahoo.com/news/stock-split-watch-apple-next-131000148.html',
 'https://finance.yahoo.com/news/apple-stock-200-time-buy-101000436.html',
 'https://finance.yahoo.com/news/spotify-streaming-giant-might-outwitted-073000149.html',
 'https://finance.yahoo.com/news/earn-500-month-apple-stock-150135356.html',
 'https://finance.yahoo.com/news/other-apple-stock-pays-monthly-164700228.html',
 'https://finance.yahoo.com/news/expect-future-earnings-growth-apple-104743921.html',
 'https://finance.yahoo.com/news/apples-services-business-is-a-bright-spot-in-the-companys-rough-2024-bofa-165556447.html',
 'https://finance.yahoo.com/news/apple-suffers-second-rating-downgrade-160958652.html',
 'https://finance.yahoo.com/news/last-time-apple-spent-much-124500620.html',
 'https://finance.yahoo.com/news/apples-generative-ai-plans-are-suddenly-a-lot-more-important-to-its-future-201956609.html',
 'https://finance.yahoo.com/news/apple-loses-top-spot-china-041348028.html',
 'https://finance.yahoo.com/news/investors-see-microsofts-stock-market-080338838.html',
 'https://finance.yahoo.com/news/2-stocks-more-magnificent-apple-130000764.html',
 'https://finance.yahoo.com/news/decoding-apple-inc-aapl-strategic-050307726.html',
 'https://finance.yahoo.com/news/apples-vison-pro-could-send-121000855.html',
 'https://finance.yahoo.com/news/4-things-know-rush-buy-171500757.html',
 'https://finance.yahoo.com/news/apple-stock-buy-142612693.html',
 'https://finance.yahoo.com/news/much-apple-shares-worth-5-163555061.html',
 'https://finance.yahoo.com/news/stock-market-news-today-us-stocks-dip-to-start-new-year-as-apple-slips-121819598.html',
 'https://finance.yahoo.com/news/where-apple-stock-3-years-153000849.html',
 'https://finance.yahoo.com/news/apple-stock-buy-sell-hold-185845526.html',
 'https://finance.yahoo.com/news/apple-stock-buy-now-124500336.html',
 'https://finance.yahoo.com/news/where-apple-stock-5-years-145441853.html',
 'https://finance.yahoo.com/news/warren-buffett-says-steve-jobs-192312997.html',
 'https://finance.yahoo.com/news/stock-better-investment-2024-apple-150049724.html',
 'https://finance.yahoo.com/news/microsoft-overtakes-apple-worlds-most-144440043.html',
 'https://finance.yahoo.com/news/congo-questions-apple-over-knowledge-135136596.html',
 'https://finance.yahoo.com/news/apple-tumbles-warning-dull-holiday-102830278.html',
 'https://finance.yahoo.com/news/apple-aapl-stock-moves-0-214518180.html',
 'https://finance.yahoo.com/news/zacks-investment-ideas-feature-highlights-082500962.html',
 'https://finance.yahoo.com/news/apple-inc-exploration-intrinsic-value-110010571.html',
 'https://finance.yahoo.com/news/greggs-bitcoin-tesla-apple-trending-tickers-111022526.html',
 'https://finance.yahoo.com/news/huawei-profit-surges-takes-share-080927855.html',
 'https://finance.yahoo.com/news/zacks-investment-ideas-feature-highlights-110000656.html',
 'https://finance.yahoo.com/news/hold-apple-aapl-long-term-123749460.html',
 'https://finance.yahoo.com/news/apple-best-magnificent-seven-stock-113000731.html',
 'https://finance.yahoo.com/news/dare-invest-apple-aapl-064949251.html',
 'https://finance.yahoo.com/news/apple-stock-down-10-5-224048496.html',
 'https://finance.yahoo.com/news/5-apple-products-avoid-buying-140009943.html',
 'https://finance.yahoo.com/news/apple-signal-ai-intent-unleashes-114945606.html',
 'https://finance.yahoo.com/news/trending-stock-apple-inc-aapl-130018800.html',
 'https://finance.yahoo.com/news/apple-stock-could-notch-record-194747011.html',
 'https://finance.yahoo.com/news/2-2-billion-reasons-buy-144500461.html',
 'https://finance.yahoo.com/news/apples-glass-supplier-corning-outperforms-194542581.html',
 'https://finance.yahoo.com/news/too-buy-apple-stock-110900320.html',
 'https://finance.yahoo.com/news/where-apple-stock-2025-090000324.html',
 'https://finance.yahoo.com/news/forget-apple-buy-unstoppable-growth-150000634.html',
 'https://finance.yahoo.com/news/unfortunate-news-meta-stock-apple-093000537.html',
 'https://finance.yahoo.com/news/analyst-upgrades-apple-stock-earnings-185619605.html',
 'https://finance.yahoo.com/news/apple-stock-top-holding-tech-120000467.html',
 'https://finance.yahoo.com/news/apples-first-quarter-has-felt-more-like-an-entire-bad-year-201715880.html',
 'https://finance.yahoo.com/news/apple-trillion-dollar-stock-2035-100000749.html',
 'https://finance.yahoo.com/news/apples-vision-pro-lands-as-the-company-faces-threats-from-all-sides-225650277.html',
 'https://finance.yahoo.com/news/apple-amazon-earnings-tech-stocks-100000361.html',
 'https://finance.yahoo.com/news/apple-stock-soar-36-gears-232719784.html',
 'https://finance.yahoo.com/news/apple-aapl-fell-restrictions-china-022600335.html',
 'https://finance.yahoo.com/news/apple-stock-slips-as-company-prepares-to-halt-apple-watch-sales-224142282.html',
 'https://finance.yahoo.com/news/happen-warren-buffett-bailed-apple-172823491.html',
 'https://finance.yahoo.com/news/buy-apple-aapl-stock-ahead-164700880.html',
 'https://finance.yahoo.com/news/stock-market-today-nasdaq-surges-as-chipmakers-shine-apple-pops-210312922.html',
 'https://finance.yahoo.com/news/too-buy-apple-stock-101500323.html',
 'https://finance.yahoo.com/news/apple-inc-aapl-ceo-timothy-055354884.html',
 'https://finance.yahoo.com/news/apple-named-top-pick-2024-150637860.html',
 'https://finance.yahoo.com/news/american-politicians-buying-10-ai-215904213.html',
 'https://finance.yahoo.com/news/apple-stock-sinks-despite-earnings-beat-as-china-sales-slow-144509608.html',
 'https://finance.yahoo.com/news/trending-stock-apple-inc-aapl-140010235.html',
 'https://finance.yahoo.com/news/sun-set-apples-innovation-driven-142100071.html',
 'https://finance.yahoo.com/news/its-not-all-doom-and-gloom-for-apple-in-2024-202332782.html',
 'https://finance.yahoo.com/news/buy-apple-inc-nasdaq-aapl-110103890.html',
 'https://finance.yahoo.com/news/apple-strengthens-supply-chain-china-152635915.html',
 'https://finance.yahoo.com/news/apple-stock-faces-three-downgrades-but-not-everyone-is-down-on-the-company-192846428.html',
 'https://finance.yahoo.com/news/apple-stock-nasdaq-aapl-2-055048576.html',
 'https://finance.yahoo.com/news/apple-stock-27-upside-according-171259933.html',
 'https://finance.yahoo.com/news/analyst-revamps-apple-stock-rating-132103626.html',
 'https://finance.yahoo.com/news/apple-aapl-beat-estimates-again-161014112.html',
 'https://finance.yahoo.com/news/apple-aapl-registers-bigger-fall-214522452.html',
 'https://finance.yahoo.com/news/apple-stock-31-upside-according-133313699.html',
 'https://finance.yahoo.com/news/apple-aapl-falls-more-steeply-214518929.html',
 'https://finance.yahoo.com/news/apple-stock-27-upside-according-201112216.html',
 'https://finance.yahoo.com/news/apple-aapl-advances-underperforms-market-224518956.html',
 'https://finance.yahoo.com/news/apple-looks-to-southeast-asia-india-as-hedge-against-china-difficulties-185416388.html',
 'https://finance.yahoo.com/news/2-reasons-buy-apple-stock-073000374.html',
 'https://finance.yahoo.com/news/prediction-warren-buffetts-second-biggest-095000279.html',
 'https://finance.yahoo.com/news/apple-aapl-rises-higher-market-224513451.html',
 'https://finance.yahoo.com/news/apple-ceo-tim-cook-much-170220216.html',
 'https://finance.yahoo.com/news/apple-earnings-time-bite-192900665.html',
 'https://finance.yahoo.com/news/apple-stock-buy-221100899.html',
 'https://finance.yahoo.com/news/paypal-raises-annual-outlook-expands-153301289.html',
 'https://finance.yahoo.com/news/google-apple-now-threatened-us-194925838.html',
 'https://finance.yahoo.com/news/apple-stock-nasdaq-aapl-could-055803122.html',
 'https://finance.yahoo.com/news/apple-aapl-ascends-while-market-224522610.html',
 'https://finance.yahoo.com/news/earn-500-month-apple-stock-150245785.html',
 'https://finance.yahoo.com/news/zacks-market-edge-highlights-aapl-083500150.html',
 'https://finance.yahoo.com/news/earn-500-month-apple-stock-170539964.html',
 'https://finance.yahoo.com/news/where-apple-stock-3-years-075900640.html',
 'https://finance.yahoo.com/news/apple-ipad-hit-eu-digital-094500468.html',
 'https://finance.yahoo.com/news/meet-unstoppable-stock-just-joined-151500438.html',
 'https://finance.yahoo.com/news/buy-apple-stock-earnings-204908376.html',
 'https://finance.yahoo.com/news/meta-platforms-bought-time-long-154934660.html',
 'https://finance.yahoo.com/news/where-apple-stock-3-years-075000500.html',
 'https://finance.yahoo.com/news/wall-street-analyst-warns-apple-163014714.html',
 'https://finance.yahoo.com/news/apple-renews-talks-openai-iphone-224431419.html',
 'https://finance.yahoo.com/news/apples-earnings-power-once-again-210654562.html',
 'https://finance.yahoo.com/news/down-over-10-time-buy-111500040.html',
 'https://finance.yahoo.com/news/where-apple-stock-5-years-145441853.html',
 'https://finance.yahoo.com/news/forget-apple-think-stock-replace-110900654.html',
 'https://finance.yahoo.com/news/apple-shares-rise-3-analyst-191312617.html',
 'https://finance.yahoo.com/news/apple-aapl-expected-beat-earnings-140211938.html',
 'https://finance.yahoo.com/news/apple-aapl-advances-while-market-214520651.html',
 'https://finance.yahoo.com/news/apple-3-trillion-stock-2025-151500285.html',
 'https://finance.yahoo.com/news/why-wall-street-is-cooling-on-apple-stock-194143216.html',
 'https://finance.yahoo.com/news/not-holding-apple-aapl-affected-130012483.html',
 'https://finance.yahoo.com/news/3-apple-products-earned-loud-194822048.html']

In [52]:
cleaned_urls = clean_urls(raw_urls)
cleaned_urls

['https://finance.yahoo.com/news/apple-is-aiming-the-vision-pro-at-enterprises-140056694.html',
 'https://finance.yahoo.com/news/reddit-apple-wetherspoons-vodafone-trending-tickers-094209950.html',
 'https://finance.yahoo.com/news/hold-apple-aapl-long-term-123749460.html',
 'https://finance.yahoo.com/news/analysis-google-apple-breakups-agenda-010424313.html',
 'https://finance.yahoo.com/news/apple-best-magnificent-seven-stock-113000731.html',
 'https://finance.yahoo.com/news/apple-stock-nasdaq-aapl-vision-000011752.html',
 'https://finance.yahoo.com/news/meta-platforms-worth-more-apple-163000701.html',
 'https://finance.yahoo.com/news/dare-invest-apple-aapl-064949251.html',
 'https://finance.yahoo.com/news/apple-nasdaq-aapl-seems-debt-140013626.html',
 'https://finance.yahoo.com/news/apple-stock-down-10-5-224048496.html',
 'https://finance.yahoo.com/news/apple-stock-down-36-as-barclays-says-iphone-weakness-will-persist-220019303.html',
 'https://finance.yahoo.com/news/apples-ipados-des

In [57]:
news_df = download_news(cleaned_urls)
news_df
# news_df.to_csv("output/news.csv")

Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
I didn't get this
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Failed to get date.
Fa

,publish_date,title,body_text,url,neg,neu,pos,compound
0,2024-04-09,Apple is aiming the Vision Pro at enterprises,It’s been just over two months since Apple (AP...,https://finance.yahoo.com/news/apple-is-aiming...,0.010,0.853,0.137,0.9988
1,2024-04-29,"Apple's iPadOS subject to tough EU tech rules,...",By Foo Yun Chee\n\nBRUSSELS (Reuters) - Apple'...,https://finance.yahoo.com/news/apples-ipados-d...,0.000,0.950,0.050,0.8074
2,2024-04-28,5 Apple Products To Avoid Buying,ozgurdonmaz / Getty Images\n\nApple is a techn...,https://finance.yahoo.com/news/5-apple-product...,0.055,0.847,0.099,0.9902
3,2024-04-12,Apple Signal of AI Intent Unleashes $112 Billi...,(Bloomberg) -- The stock market has punished A...,https://finance.yahoo.com/news/apple-signal-ai...,0.043,0.847,0.110,0.9943
4,2024-04-30,Apple's Glass Supplier Corning Outperforms in ...,Apple's Glass Supplier Corning Outperforms in ...,https://finance.yahoo.com/news/apples-glass-su...,0.045,0.865,0.090,0.9741
5,2024-03-29,Huawei Profit Surges as It Takes Share From Ap...,(Bloomberg) -- Huawei Technologies Co. sustain...,https://finance.yahoo.com/news/huawei-profit-s...,0.038,0.886,0.076,0.9135
6,2024-01-15,Apple could have a new problem: Uncle Sam,Apple (AAPL) has long avoided the government-i...,https://finance.yahoo.com/news/apple-could-hav...,0.064,0.855,0.081,0.9568
7,2024-04-25,DR Congo presses Apple over minerals supply ch...,KINSHASA (Reuters) -Democratic Republic of Con...,https://finance.yahoo.com/news/dr-congo-presse...,0.087,0.896,0.017,-0.9850
8,2024-04-23,Unfortunate News for Meta Stock and Apple Stoc...,Fool.com contributor Parkev Tatevosian discuss...,https://finance.yahoo.com/news/unfortunate-new...,0.072,0.881,0.047,-0.7901
9,2024-04-29,Analyst Upgrades Apple Stock Before Earnings,Apple Inc (NASDAQ:AAPL) will step into the ear...,https://finance.yahoo.com/news/analyst-upgrade...,0.028,0.922,0.050,0.5927


In [58]:
news_df.to_csv("output/news.csv")

In [47]:
# news = pd.read_csv("output/news.csv")
# news = pd.concat([news, news_df], axis=0, ignore_index=True)
# news.to_csv("output/news.csv", index=False)